### Run the cell below and update the Mongo Host and Port Info

In [1]:
! bash ~/PySyft/scripts/dev_tools.sh

-----------------------------
Containers | Exposed Ports
-----------------------------
/canada-seaweedfs-1  49161 49160 
/canada-queue-1  49155 
/canada-redis-1  49153 
/canada-mongo-1  49156 
/canada-jaeger-1  14268 49154 


In [2]:
## Start here.

In [1]:
# Set your public ports for mongo here.

class EnvSettings:
    MONGO_HOST = "localhost"  # # change to 'localhost'
    MONGO_PORT = "49156"  # change to public port
    MONGO_USERNAME = "root"
    MONGO_PASSWORD = "example"  # nosec
    
env_settings = EnvSettings()

In [2]:
import syft as sy
from syft.core.node.new.user import UserCreate, User, UserUpdate
from syft.core.node.new.mongo_document_store import MongoCollection, MongoDocumentStore, MongoBsonObject, MongoStorePartition
from syft.core.node.new.document_store import StoreClientConfig
from syft.core.node.common.node_table.syft_object import SyftObjectRegistry
from syft.core.node.worker import Worker
from syft.core.node.new.user_stash import UserStash
from syft.core.common.uid import UID
from syft.core.node.new.document_store import UIDPartitionKey
from syft.core.node.new.user_stash import PartitionSettings, User

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/shubham/PySyft


In [3]:
mongo_client_config = StoreClientConfig(
    hostname=env_settings.MONGO_HOST,
    port=env_settings.MONGO_PORT,
    username=env_settings.MONGO_USERNAME,
    password=env_settings.MONGO_PASSWORD,
)

In [4]:
store = MongoDocumentStore(client_config=mongo_client_config)

In [5]:
from syft.core.node.new.dict_document_store import DictDocumentStore

In [6]:
dict_store = DictDocumentStore()

In [7]:
mongo_store_ser = sy.serialize(store, to_bytes=True)

In [8]:
store = sy.deserialize(mongo_store_ser, from_bytes=True)

In [9]:
user_stash = UserStash(store=store)

In [10]:
store_partition = store.partition(settings=user_stash.settings)

In [11]:
store_partition

In [12]:
desered_store_partition = sy.deserialize(sy.serialize(store_partition, to_bytes=True), from_bytes=True)

In [17]:
user_stash = sy.deserialize(sy.serialize(user_stash, to_bytes=True), from_bytes=True)

In [18]:
user_stash.partition.collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'User_index_name': {'v': 2,
  'key': [('email', 1), ('signing_key', 1), ('verify_key', 1)],
  'unique': True}}

In [19]:
new_user = UserCreate(
    name="Shubham", email="alice1@openmined.org", password="pass", password_verify="pass"
)

In [20]:
user = new_user.to(User)

In [21]:
bson_dict = user.to(MongoBsonObject)

In [22]:
bson_dict.to(UserCreate)

```python
class User:
  id: str = 49c3089eedda4e66adbe1142c4bea9a9
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$QMibVKen77LQiiSVSWoYuufSjH4Yghr1UPbjPTRQdmY8JTKchjLc6"
  salt: str = "$2b$12$QMibVKen77LQiiSVSWoYuu"
  signing_key: str = 7ff14b8ca0b90553aec08f6a61eca0b95173c018cc3b3b56553743e9681b047f
  verify_key: str = c84469a8e9eaa7efce78401c046d0b2aff3456f140b429d0ad1e6c74f3e93314
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [23]:
response = user_stash.set(user)

In [24]:
response

Ok(syft.core.node.new.user.User)

In [25]:
# #### Create indexes
# from pymongo import IndexModel, ASCENDING, DESCENDING
# user_stash.partition.db_collection.create_index([
#     ("email", ASCENDING),
#     ("verify_key", ASCENDING)
# ], unique=True)

In [26]:
search_result = user_stash.get_by_email(email="alice1@openmined.org")

In [27]:
search_result.ok() if search_result.is_ok() else search_result.err()

```python
class User:
  id: str = 49c3089eedda4e66adbe1142c4bea9a9
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$QMibVKen77LQiiSVSWoYuufSjH4Yghr1UPbjPTRQdmY8JTKchjLc6"
  salt: str = "$2b$12$QMibVKen77LQiiSVSWoYuu"
  signing_key: str = 7ff14b8ca0b90553aec08f6a61eca0b95173c018cc3b3b56553743e9681b047f
  verify_key: str = c84469a8e9eaa7efce78401c046d0b2aff3456f140b429d0ad1e6c74f3e93314
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [32]:
uid_to_update = UID.from_string("49c3089eedda4e66adbe1142c4bea9a9")

In [33]:
update_user = UserUpdate(id=uid_to_update, email="alice1@openmined.org", name="MyBob2", institution="OpenMined")

In [34]:
update_result = user_stash.update(user=update_user.to(User))

In [35]:
update_result.ok() if update_result.is_ok() else update_result.err()

```python
class User:
  id: str = 49c3089eedda4e66adbe1142c4bea9a9
  email: str = "alice1@openmined.org"
  name: str = "MyBob2"
  hashed_password: str = None
  salt: str = None
  signing_key: str = None
  verify_key: str = None
  role: str = None
  institution: str = "OpenMined"
  website: str = None
  created_at: str = None

```

In [36]:
user_stash.get_by_email(email="bob@alice.com")

Ok(None)

In [38]:
uid_to_delete = UID.from_string("49c3089eedda4e66adbe1142c4bea9a9")
response = user_stash.delete_by_uid(uid=uid_to_delete)

In [39]:
response

Ok(<class 'syft.core.node.new.response.SyftSuccess'>: ID: 49c3089eedda4e66adbe1142c4bea9a9 deleted)

### Testing the services

In [13]:
import syft as sy
from syft.core.node.new.user import UserSearch, UserCreate
from syft.core.node.worker import Worker
from syft.core.node.new.document_store import StoreClientConfig
from syft.core.common.uid import UID

In [2]:
# Set your public ports for mongo here.

class EnvSettings:
    MONGO_HOST = "localhost"  # # change to 'localhost'
    MONGO_PORT = "49156"  # change to public port
    MONGO_USERNAME = "root"
    MONGO_PASSWORD = "example"  # nosec
    
env_settings = EnvSettings()
mongo_client_config = StoreClientConfig(
    hostname=env_settings.MONGO_HOST,
    port=env_settings.MONGO_PORT,
    username=env_settings.MONGO_USERNAME,
    password=env_settings.MONGO_PASSWORD,
)

In [3]:
from syft.core.node.new.mongo_document_store import MongoStoreConfig

In [4]:
worker = Worker(store_config=MongoStoreConfig(client_config=mongo_client_config))

Starting Worker: Loving Howard - 853630896e6e4836902c184b473b95d7 [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>]


In [5]:
worker_ser = sy.serialize(worker, to_bytes=True)

In [6]:
worker_deser = sy.deserialize(worker_ser, from_bytes=True)

In [7]:
client = sy.new_login(node=worker_deser, email="info@openmined.org", password="changethis", port=8081)

Logged into Loving Howard as <info@openmined.org>


In [8]:
u_create = UserCreate(name="Shubham", email="shubh@openmined.org", password="pass", password_verify="pass")

In [9]:
client.api.services.user.create(user_create=u_create)

```python
class UserView:
  id: str = a296ba9c96e541e2b12fac43f34a6590
  email: str = "shubh@openmined.org"
  name: str = "Shubham"
  role: str = ServiceRole.GUEST
  password: str = None
  password_verify: str = None
  verify_key: str = None
  institution: str = None
  website: str = None

```

In [10]:
client.api.services.user.search(name="Shubham")

,type
0,syft.core.node.new.user.UserView


In [14]:
uid_to_delete = UID.from_string("a296ba9c96e541e2b12fac43f34a6590")
client.api.services.user.delete(uid_to_delete)

<class 'syft.core.node.new.response.SyftSuccess'>: ID: a296ba9c96e541e2b12fac43f34a6590 deleted

In [15]:
client.api.services.user.search(name="Shubham")

[]